In [1]:
import pandas as pd
house=pd.read_csv('train.csv')
target_col='SalePrice'
id_col='Id'
target=house[target_col]
total_num=len(house)

In [2]:
# drop heavy missing cols
c=house.count()
drop_cols=c[c*2<len(house)].index
drop_cols=list(drop_cols)
drop_cols+=[id_col,target_col]
house.drop(drop_cols,axis=1,inplace=True)

In [3]:
import sys
sys.path.append('../')
from util import *

In [22]:
from sklearn.preprocessing import Imputer, StandardScaler, LabelBinarizer,LabelEncoder
from sklearn.pipeline import Pipeline, FeatureUnion

summary=house.describe()
num_imputer=Imputer(strategy='median')
# p=Pipeline([
#     ('select',DataFrameSelecter(summary.columns)),
#     ('fill',Imputer(strategy='median')),
# ])
# p.fit(house)
num_pl=Pipeline([
    ('select',DataFrameSelecter(summary.columns)),
    ('fill',Imputer(strategy='median')),
    ('scale',StandardScaler()),
])
num_pl.fit(house)

cat_cols=list(set(house.columns)-set(summary.columns))
pls=[Pipeline([
    ('select',DataFrameSelecter2(c)),
    ('drop',NanDropper()),
    ('encode',LabelBinarizer()),
]) for c in cat_cols ]

[pls[i].fit_transform(house) for i in range(len(cat_cols))]

cat_pls=[(cat_cols[i],Pipeline([
    ('select',DataFrameSelecter3(cat_cols[i])),
    ('encode',pls[i].named_steps['encode']),
])) for i in range(len(cat_cols))]

full_pl=FeatureUnion(transformer_list=[('num',num_pl)]+cat_pls)

# full_pl=FeatureUnion(transformer_list=cat_pls)
prepared_house=full_pl.transform(house)


1460

In [24]:
from sklearn import linear_model
reg=linear_model.LinearRegression()

from math import sqrt
from sklearn.model_selection import cross_val_score
reg.fit(prepared_house,target)
scores=cross_val_score(reg,prepared_house,target,scoring='neg_mean_squared_error',cv=10)
print(sqrt(-scores.mean()))

570774997965484.2


In [32]:
house_test=pd.read_csv('test.csv')
test_id=house_test[id_col]
drop_cols.pop()
house_test.drop(drop_cols,axis=1,inplace=True)
prepared_house_test=full_pl.transform(house_test)

In [33]:
prices=reg.predict(prepared_house_test)
df=pd.DataFrame({'Id':test_id,'SalePrice':prices})
df.to_csv('result.csv',index=False)

kaggle score: 0.25832, rank: 1666

In [25]:
#use rf reg
from sklearn.ensemble import RandomForestRegressor
reg=RandomForestRegressor()

from math import sqrt
from sklearn.model_selection import cross_val_score
reg.fit(prepared_house,target)
scores=cross_val_score(reg,prepared_house,target,scoring='neg_mean_squared_error',cv=10)
print(sqrt(-scores.mean()))

31455.31597592617


In [29]:
prices=reg.predict(test_data)
df=pd.DataFrame({'Id':test_id,'SalePrice':prices})
df.to_csv('result.csv',index=False)

kaggle score: 0.16006, rank: 1360

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
reg=GradientBoostingRegressor()

from math import sqrt
from sklearn.model_selection import cross_val_score
reg.fit(prepared_house,target)
scores=cross_val_score(reg,prepared_house,target,scoring='neg_mean_squared_error',cv=10)
print(sqrt(-scores.mean()))

26671.074237621673


In [34]:
prices=reg.predict(prepared_house_test)
df=pd.DataFrame({'Id':test_id,'SalePrice':prices})
df.to_csv('result.csv',index=False)

kaggle score: 0.14196, rank: 1116

encode all category cols

kaggle score: 0.13303, rank: 877

In [7]:
# # check cols have missing values
# total_num=len(house)
# missing_value_cols=list(filter(lambda c: house[c].count()<total_num, house.columns))
# # calc missing percent
# missing_percents=list(map(lambda c:((total_num-house[c].count())*100/total_num,c),missing_value_cols))
# heavy_missings=list(filter(lambda x:x[0]>50, missing_percents))
# heavy_missings.sort(reverse=True)
# heavy_missings

In [8]:
# drop these heavy-missing cols
# [house.drop(x[1],axis=1,inplace=True) for x in heavy_missings]
# drop_missing_cols=[x[1] for x in missing_percents]
# house.drop(drop_missing_cols,axis=1,inplace=True)
# print(len(house.columns))

In [9]:
%matplotlib inline
#plt=house.hist(figsize=(20,15))